# 전처리

- 편의를 위해 함수 이름은 ```def (내용)_check():``` 모양으로 작성하였습니다.

## 라이브러리

In [4]:
# 정규식
import re

# 특수문자(이모티콘)제거
from emoji import core # 정규식으로 없어지지 않는 특수문자 제거

# 형태소 분석기
from konlpy.tag import Kkma, Hannanum, Mecab, Okt # Mecab은 error발생으로 linux환경에서 실행 필요.

# 형태소 분석기 : Komoran
from PyKomoran import Komoran, DEFAULT_MODEL # Komoran error는 조금 더 살펴보겠음...

import pandas as pd

In [5]:
Kkma = Kkma()
Hannanum = Hannanum()
Okt = Okt() # 구 Twitter
komoran = Komoran(DEFAULT_MODEL['FULL']) # FULL or LIGHT

In [6]:
Mecab = Mecab()

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [18]:
df =pd.read_parquet('./data/crwaling_data4.parquet')
df.head()

,text,date,click,topic
0,어떤게나을까요?! 지금은 웅진 학습지로만(지면) 하고있어요~ 패드로 슬슬 갈까하는데...,2023-03-03 14:49:00,502,초1
1,어떤게나을까요?! 지금은 웅진 학습지로만(지면) 하고있어요~ 패드로 슬슬 갈까하는데...,2023-03-03 14:49:00,502,초1
2,초1이고 밀크티 하고 있어요\n오늘의 학습만 매일하고 있는데요\n초등 2학년때까진 ...,2023-03-20 22:35:00,208,초1
3,"대부분의 받아쓰기는 빠르면 4살, 늦어도\n초등학교 입학전부터 학습하는 경우가 많잖...",2022-09-06 16:58:00,661,초1
4,"초1인데 패드로 하는 학습지를 시켜보고 싶은데요.\n빨간펜, 윙크,밀크티 중 어떤게...",2023-03-14 15:02:00,219,초1


In [19]:
df.text[0]

'어떤게나을까요?! 지금은 웅진 학습지로만(지면) 하고있어요~ 패드로 슬슬 갈까하는데 고학년까지 과학이랑 영어같이하면 구몬이 나을지,밀크티가 나을지 모르겠어요~ 비교좀 부탁드려요~'

### 전처리 과정

<details>
<summary>내용</summary>
<div markdown="1">       
1. 태그 제거
    - `http`, `제목 : ` 등의 규칙적이게 나타나는 값 제거

1. 특수문자 제거
    - 정규식(re)
    - 이모티콘(emoji)

1. 문자 통일
    - 영어 대소문자
    - 동음이의어 및 같은 의미를 담는 문자
        - ex) 초2, 초등2학년, 초등학교 2학년
        - ex) 밀크티, 밀크T, 밀ㅋㅌ, ㅁㅋㅌ, 밀*티, MilkT

1. 불용어 제거
    - stop_words
        - mecab에는 불용어사전 기능이 존재함.
        - 다른 형태소분석기의 경우에는 `stopwords`리스트와 `if not in` 을 활용
        - 인터넷에 한국어 stop_words를 공유하는 경우가 있으므로 활용

1. 특정 문자 존재 여부
    - Keyword : `밀크티`, `홈런`, `태블릿` 등등...

1. 토큰화
    - Konlpy

1. 표제어 추출
    - LDA등의 모델 활용

1. 벡터화
    - numpy를 사용하여 벡터화 - 모델링에 활용

1. 글자 수
    - 글자수가 너무 긴 경우 : 내용이 부적합할 수 있다. (ex. 해피빈 모금함 후원자명단 이라는 글이 크롤링 되었는데, 아이디와 닉네임을 나열한 글이라서 더미데이터이면서 글자 수가 매우 길었다.)
    - 글자수가 너무 짧은 경우 : 내용이 담겨있지 않을 수 있다. (ex. 밀크티 저는 비추천이요)

1. 글자 형태
    - 같은 문장이 여러번 쓰이는 경우 : 광고성 글일 가능성이 있다. (XX동 XX학원 XXX선생입니다. 저희 학원에서는~)

1. 품사 태깅
    - Konlpy.pos를 통해 불필요한 품사(조사 등)을 제거
        - 체언(명사,대명사, 수사)만 남기고 제거, 필요에 따라 용언(중 동사, 형용사)도 남김.

1. 띄어쓰기 구분
    - 한글, 특히 온라인에 게시된 글들은 띄어쓰기가 올바르게 되어있지 않을 확률이 높기에 구분 짓는 방법 필요
        - Mecab이나 Hannanum등의 형태소 분석기는 띄어쓰기를 잘 구분하지 못한다.

1. 분석에 불필요한 문장 제거
    - ?!?!?!?!?!?!?! - 해봐야 알 듯 해서 아직 미작성

1. 낚시성 글 제거
    - 이 프로젝트의 경우 크롤링 대상이 카페에서 이루어지므로 낚시성 글일 확률이 낮고, 주관성이 들어간 글들을 수집하기에 이 단계는 진행하지 않는다.

1. 기준 설정
    1. 기간 : 최근 n개월~
    1. 광고 : 
        - 광고 시 들어가는 문구인 `본 포스팅(이 글)은 해당 업체로부터 제품&원고료를 지원받아 작성한 리뷰입니다.`의 존재 여부 확인
            - 이미지로 존재할 경우, 이미지 text화(library : tesseract)를 통해 확인

    1. 키워드 : 
        1. 검색 시 키워드 설정
            - 초등 `@@@` :` @@@` = [`태블릿`, `온라인 학습`, `자기주도학습`, `선행학습`, ...]
            - `@@@` 밀크티 : `@@@` = [`초등`, `중등`, ...]

        1. 분석&시각화 키워드 설정
            - 주제에 부합한 키워드(하이퍼파라미터.. HP) 설정
                - `컨텐츠`가 주제일 경우 HP를 ['게임', '독서', ...]로 설정
        
        1. 중요도 & 긍/부정 여부
            - 중요도 : LDA등의 토픽 모델
            - 긍/부정 : Sentiment_Analysis
            - 아직 실습해보지 못해서 잘 모름.
</div>
</details>

1. tag제거
    - 'http'나 text에서 규칙적으로 나오는 불필요한 값 제거

In [34]:
def del_tag_check(text):
    text = re.sub('http:', '', text)
    text = re.sub('comhttpsm.', '', text)
    text = re.sub(r'(\d{2,3})-(\d{3,4})-?(\d{0,4})?', '', text) # 전화번호 제거
    text = re.sub('blog.', '', text)
    text = re.sub('naver.', '', text)
    text = re.sub('co. kr', '', text)

    return text

2. 특수문자 제거
    - 정규식(re)
    - 이모티콘(emoji)
    - 자음만 존재하는 경우
        - ex. 'ㅎㅎ맞아요'
        - but 인터넷 텍스트에서는 자음으로 줄임말을 쓰는 경우가 있으니, 2단계(문자 통일)이후 진행

In [20]:
# 특수 기호 제거
df.text = df.text.apply(lambda x : re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n★▲;]','', x))

# '\E' 모양 제거
df.text = df.text.apply(lambda x : re.sub('[\a-zA-Z]. ','',x))

# 이모티콘 제거
df.text = df.text.apply(lambda x : core.replace_emoji(x, replace=''))

# # 자음만 적힌 경우 제거
# df.text = df.text.apply(lambda x : re.sub('[ㄱ-ㅎ]+', '', x))

display(df.head())
display(df.text[0])

,text,date,click,topic
0,어떤게나을까요 지금은 웅진 학습지로만지면 하고있어요 패드로 슬슬 갈까하는데 고학년까...,2023-03-03 14:49:00,502,초1
1,어떤게나을까요 지금은 웅진 학습지로만지면 하고있어요 패드로 슬슬 갈까하는데 고학년까...,2023-03-03 14:49:00,502,초1
2,초1이고 밀크티 하고 있어요오늘의 학습만 매일하고 있는데요초등 2학년때까진 이걸로 ...,2023-03-20 22:35:00,208,초1
3,대부분의 받아쓰기는 빠르면 늦어도초등학교 입학전부터 학습하는 경우가 많잖아요.하지만...,2022-09-06 16:58:00,661,초1
4,초1인데 패드로 하는 학습지를 시켜보고 싶은데요.빨간펜 윙크밀크티어떤게 좋을까요빨간...,2023-03-14 15:02:00,219,초1


'어떤게나을까요 지금은 웅진 학습지로만지면 하고있어요 패드로 슬슬 갈까하는데 고학년까지 과학이랑 영어같이하면 구몬이 나을지밀크티가 나을지 모르겠어요 비교좀 부탁드려요'

In [22]:
# 이모티콘 제거 예시
test_text = '이모티콘 실험이에용😊😊👍👍'

# re 사용
display(re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n★▲;]','', test_text))

# emoji 사용
display(core.replace_emoji(test_text, replace=''))

'이모티콘 실험이에용😊😊👍👍'

'이모티콘 실험이에용'

3. 문자 통일
    - 영어 대소문자
    - 동음이의어 및 같은 의미를 담는 문자 - `하나씩 추가 해야될 듯`
        - ex) 초2, 초등2학년, 초등학교 2학년
        - ex) 밀크티, 밀크T, 밀ㅋㅌ, ㅁㅋㅌ, 밀*티, MilkT

        - 현재 문자통합 상황 : 
            `밀크티, 홈런`

In [26]:
# 영어 소문자화
df['text'] = df['text'].apply(lambda x: x.lower())

# 같은 의미를 담은 문자
def combine_check(contents):

    # 밀크티
    contents = re.sub('밀크t', '밀크티', contents) #영어를 모두 소문자화 했으므로 대문자T는 진행 X
    contents = re.sub('밀ㅋㅌ', '밀크티', contents)
    contents = re.sub('ㅁㅋㅌ', '밀크티', contents)
    contents = re.sub('밀*티', '밀크티', contents)
    contents = re.sub('밀크*', '밀크티', contents)
    contents = re.sub('*크티', '밀크티', contents)
    contents = re.sub('밀**', '밀크티', contents)
    contents = re.sub('*크*', '밀크티', contents)
    contents = re.sub('**티', '밀크티', contents)
    contents = re.sub('milkt', '밀크티', contents)

    # 홈런
    contents = re.sub('홈*', '홈런', contents)
    contents = re.sub('*런', '홈런', contents)
    contents = re.sub('아이스크림 홈런', '홈런', contents)
    contents = re.sub('아이스크림 에듀', '홈런', contents)
    contents = re.sub('홈런 홈런', '홈런', contents) # '아이스크림 에듀 홈런' 으로 쓰였을 경우 홈런이 두번...
    contents = re.sub('ㅎㄹ', '홈런', contents)
    contents = re.sub('ㅎㄹ', '홈런', contents)

    # 명사
    contents = re.sub('할미','할머니', contents)
    contents = re.sub('티비','텔레비전', contents)
    contents = re.sub('','', contents)
    contents = re.sub('','', contents)
    contents = re.sub('','', contents)

    # contents = re.sub('','', contents)

4. 불용어 제거
    - stop_words
        - mecab에는 불용어사전 기능이 존재함.
        - 다른 형태소분석기의 경우에는 `stopwords`리스트와 `if not in` 을 활용
        - 인터넷에 한국어 stop_words를 공유하는 경우가 있으므로 활용

In [24]:
# 1. Mecab

In [28]:
# 그 외의 분석기 (리스트는 예시)
stop_words = ['아이', '맘', '거', '저', '저희', '것', '학모', '감사', '게', '때', '애', '원', '제', '개', '일', '듯', '만', '번', '데', '분', '건', '정도', '달', '정보', '이번', '부탁', '걸로', '후', '나', '부분', '걸', '저흰', '답변', '마음', '뼈', '건가요', '오늘', '왜', '시', '꺼', '친구', '마미', '때문', '하루', '명', '돈', '해요', '이', '가요', '기', '지금', '건가요', '반', '주세요', '편', '외', '소개', '입니다', '곳', '이거', '뭐', '땐', '건지', '그거', '점', '땐', '미', '풀', '대', '안', '눈', '공주', '사', '오', '백', '후', '나']

def stopwords_check(A):
    text_list = []
    if A not in stop_words:
        text_list.append(A)

    return text_list

5. 특정 문자 존재 여부
    - Keyword : `밀크티`, `홈런`, `태블릿` 등등...

In [30]:
# keyword가 text에 존재하는지 여부확인

def keyword_check(keyword, text):
    if keyword in text:
        return text

    else : 
        # none이나 ''이면 전처리 시 불편해서 우선은 이렇게 해뒀습니다.
        return f'None keyword : {text}' 

6. 토큰화
    - Konlpy

In [36]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

text = df['text'][0]
print(tokenizer.tokenize(text))

['어떤게나을까요', '지금은', '웅진', '학습지로만지면', '하고있어요', '패드로', '슬슬', '갈까하는데', '고학년까지', '과학이랑', '영어같이하면', '구몬이', '나을지밀크티가', '나을지', '모르겠어요', '비교좀', '부탁드려요']


7. 표제어 추출
    - LDA등의 모델 활용
    
    `codefile 참조`

8. 벡터화
    - numpy를 사용하여 벡터화 - 모델링에 활용

모델링 파트에서<br>
<code>
np.arange(len(top_milk_nouns)) </code><br><br>혹은<br> <code>milk_indexes = df[df['cluster'] == 0].index<br>
similarity = cosine_similarity(tfidf_vectors[milk_indexes[0]], tfidf_vectors[milk_indexes])<br>
milk_1_similarity = np.sqrt(similarity.reshape(-1)[::-1]) <br> 
</code>
같은 모양으로 각 모델에 맞게 벡터화 시킴

9. 글자 수
    - 글자수가 너무 긴 경우 : 내용이 부적합 할 수 있다.
        - ex. 크롤링한 데이터에 '해피빈 모금함 후원자 명단' 이라는 글이 있었는데, 아이디와 닉네임을 단순 나열한 글이였다. 이 글 안에 전처리시 사용한 키워드들이 모두 포함되어 있어서 삭제되지 않았음.

    - 글자수가 너무 짧은 경우 : 내용이 담겨있지 않을 수 있다.
        - ex. "밀크티 저는 비추천이요~" 
            - 비추천하는 이유가 담겨있지 않음.
            - 단순 긍/부정적 키워드 판별에는 쓰일 수 있음.

In [35]:
def length_check(text):
    if len(text) < 10:
        return 'Error : 문장짧음'

    elif len(text) > 100000:
        return 'Error : 문장이 너무 김'

    else:
        return text

10. 글자 형태
    - 같은 문장이 여러번 쓰이는 경우 : 광고성 글일 가능성이 있다. (XX동 XX학원 XXX선생입니다. 저희 학원에서는~)

    ```문장 유사도 검출 함수로 사용하면 될듯 어떤거 쓸지는 아직 못정함...```<br>
    ```해당 함수를 사용하여 중복 글 제거에도 사용할 수 있음```

11. 품사 태깅
    - Konlpy.pos를 통해 불필요한 품사(조사 등)을 제거
        - 체언(명사,대명사, 수사)만 남기고 제거, 필요에 따라 용언(중 동사, 형용사)도 남김.

In [ ]:
# 주어 찾기 함수

def subject_extraction_check(text):
    
    # 형태소 분석
    pos = Kkma.pos(text)
    # pos = Mecab.pos(text)
    # pos = Okt.pos(text)
    # pos = Hannanum.pos(text)
    # pos = Komoran.pos(text)

    # 주어 추출
    for i in range(len(pos)):
        if pos[i][1] == 'NNG':  # 명사
            if i == 0 or (i > 0 and pos[i-1][1] != 'NNG'):  # 주어는 문장 첫 단어이거나 앞 단어가 조사가 아닌 경우
                return pos[i][0]

In [ ]:
# 명사, 동사 추출
# dictionary를 받아서 dictionary로 리턴해줌
# side project에서 '밀크티'와 '홈런' 비교를 하느라 이렇게 만들어서 참고해주세요

def extract_check(dict): 
    # dictionary 형태를 받아서 key값을 받아옴
    key1 = list(dict.keys())[0]
    key2 = list(dict.keys())[1]

    key1_list = []
    key2_list = []

    for i in range(len(dict[key1])):
        pos = Kkma.pos(dict[key1][i])
        for j in pos:
            if j[1] == 'VA': #동사
                if j[1] not in stop_words: # 이 경우는 토큰화 할 때 stop_words 사용함
                    # Kkma의 경우 용언의 의미부분만 추출해서 ('먹었어요' -> '먹'으로 추출) '다'를 붙여줌
                    key1_list.append(j[0]+'다') 
            if j[1] == 'NNG': #명사
                if j[1] not in stop_words:
                    key1_list.append(j[0])

    for i in range(len(dict[key2])):
        pos = Kkma.pos(dict[key2][i])
        for j in pos:
            if j[1] == 'VA': #동사
                if j[1] not in stop_words:
                    key2_list.append(j[0]+'다')
            if j[1] == 'NNG': #명사
                if j[1] not in stop_words:
                    key2_list.append(j[0])

    frequncy_dict = {key1 : key1_list, key2 : key2_list}

    return frequncy_dict

12. 띄어쓰기 구분
    - 한글, 특히 온라인에 게시된 글들은 띄어쓰기가 올바르게 되어있지 않을 확률이 높기에 구분 짓는 방법 필요
        - Mecab이나 Hannanum등의 형태소 분석기는 띄어쓰기를 잘 구분하지 못한다.

13. 분석에 불필요한 문장 제거
    - ?!?!?!?!?!?!?! - 해봐야 알 듯 해서 아직 미작성

14. 기준 설정
    1. 기간 : 최근 n개월~
    1. 광고 : 
        - 광고 시 들어가는 문구인 `본 포스팅(이 글)은 해당 업체로부터 제품&원고료를 지원받아 작성한 리뷰입니다.`의 존재 여부 확인
            - 이미지로 존재할 경우, 이미지 text화(library : tesseract)를 통해 확인

    1. 키워드 : 
        1. 검색 시 키워드 설정
            - 초등 `@@@` :` @@@` = [`태블릿`, `온라인 학습`, `자기주도학습`, `선행학습`, ...]
            - `@@@` 밀크티 : `@@@` = [`초등`, `중등`, ...]

        1. 분석&시각화 키워드 설정
            - 주제에 부합한 키워드(하이퍼파라미터.. HP) 설정
                - `컨텐츠`가 주제일 경우 HP를 ['게임', '독서', ...]로 설정
        
        1. 중요도 & 긍/부정 여부
            - 중요도 : LDA등의 토픽 모델
            - 긍/부정 : Sentiment_Analysis
            - 아직 실습해보지 못해서 잘 모름.

--------------
진행중~

### 키워드

<컨텐츠>
게임 : 게임이라는 인식의 글이 많음. 아이 입장에서는 게임에 관심, 부모님은 내키지 않음
	- 밀크티 홈페이지에 오각형 parameter(스토리,멀티미디어,마인드맵,동영상강의,게임)으로 나타나 있어서 고학년으로 갈 수록 게임의 비중이 낮아지는 걸 나타내지만 부모님들은 당장이 싫은건가...? 그럼 왜 필요한지...? 음...

교재 : 무료 체험 시 교재가 오지 않아서 소비자가 교재에 대한 정보획득이 어렵다.

양 : 컨텐츠의 절대적인 양과 사용 가능한 컨텐츠(밀크T나 윙크의 경우엔 다음 차시 학습이 잠겨있다.)

<과목>
영어 : 애니메이션의 양 + 언어 선택의 유무, 

독서 : 

<보상>
보상 : 밀크티 : 컵(기프티콘 교환)

<평가>

<난이도>
문제 : 문제출제의 기준이 명확하게

<캐릭터>
캐릭터 : 아이들이 좋아할만 하게 귀엽다, 너무 도배되어있는 느낌이라 산만하다.

<학습>
자기주도 : 성향에 따라서 선택을 다르게 하라는 조언. 능동적인 아이는 효용성이 높지만, 그렇지 않은 아이들은 부모님이 옆에서 케어하기 힘들어함

담임교사 : 학생의 케어에 대한 이야기. 진도의 속도 등을 정해주거나 조언해주는 등의 케어에 만족하는 경우가 있음.

무료체험 : 무료체험이 의사선택에 가장 큰 영향을 끼침. - 여기엔 모든 키워드가 포함될듯
	- 혜택?

오답 : 문제에 틀리면 오답이라는 소리가 부모님 입장에서는 별로라고 생각하는 경우가 있다. 홈런의 경우에는 바로 복습...? 이건 확인을 해보자

<기기>
기기 : 갤럭시탭을 이용하기에 학습이 끝나도 태블릿의 효용성을 장점으로 꼽음. 다만 이러한 장점이 아이가 학습을 하지않고 유튜브를 보는 등의 집중력 저하에 영향을 끼칠 수 있다는 단점도 존재함.

성능 : 오류발생 - 자잘한 딜레이 혹은 서버 접속 오류

가격 : 강제로 비싼 값에 구매.

<지속성>

약정 : 약정에 대한 부담감이 있음. 밀크T의 경우 위약금이 없고(...?!) 기기 할부금만 내면 된다는 장점이 있음.





그 외...
<밀크T 중학>
	고등 입시 정보 게시
	교재(It북) 긍정적 평가


---------------------

### 모델 파트 작성
[[모델 및 시각화]]

NLP : 문장 내 특정 위치에 출현하기 적합한 단어 예측 모델
(ex. 초등 태블릿 학습을 고민해보려고 밀크티 무료체험을 해보니 컨텐츠가 풍부하고 아이가 즐거워해서 밀크티 학습을 ??? 하기로 했다.)
	- 기계 번역 : 자연스런 문장 생성
	- 오타 교정
	 - 음성 인식
	 - 검색어 추천

Word Cloud : 출현 빈도가 높은 키워드 들을 구름같이 시각적으로 나타내줌

Treemap : 여러개의 직사각형으로 크기와 색의 진한 정도에 따라서 데이터의 양을 나타내준다.


SNA(Social Network Analysis : 사회 연결망 분석) : 
	객체들 간의 관계를 시각화 표현하여 구조를 파악하기 용이하다.

Cluster(군집화) : 객체(문장)의 특징이 유사한 그룹으로 군집화하여 시각화
	- Dendrogram : 유사한 특징을 가진 객체들끼리 토너먼트 모양의 선으로 연결

TF-IDF : 단어의 빈도 수를 이용한 수치화 방법
DTM(Document-Term Matrix) : 단어들의 빈도를 행렬로 표시한 것
LSA(Latent Semantic Analysis : 잠재 의미 분석) : 
	DTM, TF-IDF는 단어의 빈도 수를 이용한 수치화 방법이기 때문에 단어의 의미를 고려하지 못한다는 단점이 있는데 이를 보완하여 DTM의 잠재된 의미를 이끌어 내는 방법
LDA(Latent Dirichlet Allocation : 잠재 디리클레 할당)(TM_W2V  : 토픽모델링) : 
	토픽의 개수를 하이퍼파라미터로 지정하여 해당 개수만큼의 토픽이 문장에 존재할 확률을 예측한다.

Sentiment Analysis(감정 분석): 텍스트가 긍정적인지, 부정적인지 혹은 중립적인지 확인
...오?!

